In [2]:
from gensim import models
import numpy as np
import functions as fc

In [8]:
import io
from tqdm import tqdm
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in tqdm(fin):
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data
data = load_vectors('cc.ko.300.vec')

FileNotFoundError: [Errno 2] No such file or directory: 'cc.ko.300.vec'

In [6]:
print(len(data))

NameError: name 'data' is not defined

In [7]:
import os
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm

if __name__ == '__main__':
    embeddings_index = {}

    f = open(os.path.join('cc.ko.300.vec'), 'r', encoding='utf-8')
    def loading(line):
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        return word, coefs

    embeddings_index = dict(Parallel(n_jobs=-1)(delayed(loading)(line) for line in f))
    f.close()
    print(len(embeddings_index))

exception calling callback for <Future at 0x2429f6c9438 state=finished returned list>
Traceback (most recent call last):
  File "C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\joblib\parallel.py", line 808, in dispatch_one_batch
    tasks = self._ready_batches.get(block=False)
  File "C:\Users\YOON\anaconda3\envs\nlp\lib\queue.py", line 161, in get
    raise Empty
queue.Empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\joblib\parallel.py", line 347, in __call__
    self.parallel.dispatch_next()
  File "C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\joblib\parallel.py", line 780, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\joblib\

1173479


In [7]:
embeddings_index['짜증나다']

NameError: name 'embeddings_index' is not defined

In [9]:
import pickle
import pandas as pd
df_train = pd.read_pickle('token_train_data.pkl')
df_test = pd.read_pickle('token_test_data.pkl')

token_train_data, train_lable = df_train['tokens'].values.tolist(), df_train['labels']
token_test_data, test_lable = df_test['tokens'].values.tolist(), df_test['labels']



In [15]:
simple_ko_vec = {}
oov_list=[]
for sent in token_train_data+token_test_data:
    for token in sent:
        try:
            simple_ko_vec[token] = ko_model.wv.word_vec(token)
        except:
            oov_list.append(token)
oov_list= list(set(oov_list))
print(len(oov_list))
print(len(simple_ko_vec))

0
45504


In [16]:
#필요단어 선별하여 pickle에 딕셔너리 저장 (총 단어수 : 31604)
import pickle
with open('simple_ko_vec.pkl','wb') as fw:
    pickle.dump(simple_ko_vec, fw)

In [17]:
with open('simple_ko_vec.pkl','rb') as fw:
    a= pickle.load(fw)

In [10]:
# 불러들이는데 시간 오지게 걸리고? 별로 안걸리는데?, 다운 받아서 쥬피터 폴더에 넣어야함
ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [ ]:
from konlpy.tag import Okt # 이걸 써서 포스 태깅을 할꺼고
okt = Okt()
import pandas as pd # 이것들 가져와
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

In [11]:
#문장들 전처리하기
def preprocessing(data):
    data.drop_duplicates(subset=['document'], inplace=True)
    data = data.dropna(how = 'any')
    data['document'] = data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    data['document'].replace('', np.nan, inplace=True)
    data = data.dropna(how = 'any')
    sentences = data['document'].tolist()
    label = data['label']
    print('data len = {}'.format(len(sentences)))
    return sentences, label

In [12]:
# 단어 토큰화 하기
def tokenize(sentence):
    okt = Okt()
    tokenized_sentence = []

    # 우선 단어의 기본형으로 모두 살리고, 명사, 동사, 영어만 담는다.
    # 그냥 nouns로 분리하는 것보다 좀 더 정확하고 많은 데이터를 얻을 수 있다.
    for line in sentence:
        result = []
        temp_sentence = okt.pos(line, norm=True, stem=True) # 먼저 형태소 분리해서 리스트에 담고

        for i in temp_sentence:                             
            if (i[1] == 'Noun' or i[1] == 'Adjective' or i[1] == 'Alpha'):                  
                result.append(i[0])
            
        tokenized_sentence.append(result)

    return tokenized_sentence

In [13]:
def pad_sequence(sentences, padding_word="<PAD/>"): #  오른쪽을 패딩주기
    maxlen = 40
    padded_sentences = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        if len(sentence)<=maxlen:
            num_padding = maxlen - len(sentence)
            new_sentence = sentence + [padding_word] * num_padding
        else : new_sentence = sentence[:maxlen]
        padded_sentences.append(new_sentence)
    return padded_sentences

In [14]:
# ko_model.wv.word_vec('<PAD/>')
paddedarray = np.array([ ko_model.wv.word_vec(token) for x in padded for token in x])
final_array=paddedarray.reshape(-1,max_len,300) # 이런 형태네? max_length of sent = 22(34),근데 10개의 sent, 22*10 fasttext = 300(vector) 
final_array.shape

NameError: name 'padded' is not defined

In [ ]:
def vectorize(padded_sentences):
    paddedarray = np.array([ ko_model.wv.word_vec(token) for x in padded_sentences for token in x])
    final_array=paddedarray.reshape(-1,max_len,300)

In [ ]:
pre_train_data, train_label = preprocessing(train_data)
pre_test_data, test_label = preprocessing(test_data)

In [ ]:
token_train_data = tokenize(pre_train_data)
token_test_data = tokenize(pre_test_data)

In [ ]:
df['len']=[len(i) for i in df['tokens']]
import matplotlib.pyplot as plt

df['len'].hist(bins =30)

In [ ]:
## 토큰화된 데이터 pickle로 저장
# import pandas as pd
# import pickle
# df = pd.DataFrame(np.array(token_test_data),columns=['tokens'])
# df['labels']=list(map(int,test_label))
# df2 = pd.DataFrame(token_test_data,test_label,columns=['tokens','lable'])
# df1.to_pickle('token_train_data.pkl')
# df2.to_pickle('token_test_data.pkl')

In [ ]:
import pickle
import pandas as pd
df_train = pd.read_pickle('token_train_data.pkl')
df_test = pd.read_pickle('token_test_data.pkl')

token_train_data, train_lable = df_train['tokens'], df_train['labels']
token_test_data, test_lable = df_test['tokens'], df_test['labels']

In [ ]:
df_test

In [18]:
import functions as fc
max_len=30
final_train_data = fc.fasttext_vectorize(fc.pad_sequence(token_train_data), max_len=max_len)
final_test_data = fc.fasttext_vectorize(fc.pad_sequence(token_test_data), max_len=max_len)


MemoryError: Unable to allocate 2.24 GiB for an array with shape (2000000, 300) and data type float32

In [ ]:
from tensorflow import keras
import tensorflow as tf
embedding_dim = 200
filter_sizes = (3, 4, 5)
num_filters = 100
dropout = 0.5
hidden_dims = 10

batch_size = 50
num_epochs = 10
min_word_count = 1
context = 10

conv_blocks = []

sequence_length = 200

# input_shape = (sequence_length, embedding_dim) # input shape for
input_shape = (40, 300) # input shape for data, (max_length of sent, vect)

model_input = keras.layers.Input(shape=input_shape)

z = model_input
for sz in filter_sizes:
    conv = keras.layers.Conv1D(filters=num_filters,
                         kernel_size=sz,
                         padding="valid",
                         activation="relu",
                         strides=1)(z)
    conv = keras.layers.MaxPooling1D(pool_size=2)(conv)
    conv = keras.layers.Flatten()(conv)
    conv_blocks.append(conv)
z = keras.layers.Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
z = keras.layers.Dense(hidden_dims, activation="relu", kernel_regularizer=keras.regularizers.l2(0.003), bias_regularizer=keras.regularizers.l2(0.003))(z)
z = keras.layers.Dropout(dropout)(z)
model_output = keras.layers.Dense(1, activation="sigmoid")(z)

model = keras.Model(model_input, model_output)



In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
print(model.summary())

In [ ]:
import os
checkpoint_dir = './ckpt1'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=0),
    # This callback saves a SavedModel every 100 batches.
    # We include the training loss in the folder name.
    keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_dir + '/ckpt-loss={loss:.3f}',
        save_freq=500)
]
history = model.fit(final_train_data, train_lable, epochs=10, callbacks=callbacks, batch_size = batch_size, validation_data=(final_test_data, test_lable))

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_' + string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_' + string])
  plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

In [ ]:
history.epoch = list(map(lambda x: x+25, history.epoch))
history.epoch

In [ ]:
data = {'val_accuracy':[],'accuracy':[],'epoch':[],'loss':[],'val_loss':[]}

In [ ]:
for i in data.keys():
    if i == 'epoch':
        data['epoch'].extend(history.epoch)
    else:
        data[i].extend(history.history[i])

In [ ]:
b = pd.DataFrame(data)
b

In [ ]:
new=pd.concat([new,b])
new

In [ ]:
new.set_index('epoch', inplace=True)

In [ ]:
new

In [ ]:
import matplotlib.pyplot as plt
def plot_graphs(data, string):
  plt.plot(data[string])
  plt.plot(data['val_' + string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_' + string])
  plt.show()

plot_graphs(new, 'accuracy')
plot_graphs(new, 'loss')